## Rekomendacja filmów
W projekcie przedstawimy zbudowany model rekomendacji filmów, który bazuje na różnych miarach podobieństwa. Wykorzystujemy metody takie jak odległość euklidesowa, odległość cosinusowa oraz odległość chi-kwadrat, aby zaproponować filmy podobne do tego, który użytkownik wprowadził. 

Zanim przejdziemy do zbudowania modelu, zapoznajmy się z miarami, na których bazujemy.


### Miara euklidesowa

Odległość euklidesowa jest jedną z najczęściej używanych miar odległości w przestrzeni wielowymiarowej. Jest to miara prostej linii między dwoma punktami w przestrzeni.

W dwuwymiarowej przestrzeni euklidesowej (np. na płaszczyźnie), odległość $d$ między dwoma punktami $(x_1, y_1)$ i $(x_2, y_2)$ obliczamy za pomocą wzoru:

$$ d_E = \sqrt{(x_2 - x_1)^2 + (y_2 - y_1)^2} $$

Ogólnie, w przestrzeni $n$-wymiarowej, dla punktów o współrzędnych $P = (p_1, p_2, ..., p_n)$ i $Q = (q_1, q_2, ..., q_n)$, odległość Euklidesowa wyraża się wzorem:

$$ d_E(\mathbf{P}, \mathbf{Q}) = \sqrt{\sum_{i=1}^{n} (p_i - q_i)^2} $$

### Miara cosinusowa

Miara cosinusowa mierzy podobieństwo między dwoma wektorami na podstawie kąta między nimi. Jest to miara, która nie uwzględnia kierunek wektorów. Jest szczególnie użyteczna w analizie tekstów i danych wysokowymiarowych, gdzie wektory reprezentują często występowanie różnych cech.

Dla dwóch wektorów $\mathbf{A} $ i $\mathbf{B} $ w przestrzeni n-wymiarowej, podobieństwo cosinusowe jest zdefiniowane jako:

$$S_C(\mathbf{A}, \mathbf{B}) = \frac{\mathbf{A} \cdot \mathbf{B}}{\|\mathbf{A}\| \|\mathbf{B}\|} $$

Gdzie:
- $\mathbf{A} \cdot \mathbf{B}$ oznacza iloczyn skalarny dwóch wektorów,
- $\|\mathbf{A}\| $ i $\|\mathbf{B}\| $ oznaczają normę (długość) tych wektorów.

Odległość cosinusowa, dla tak zdefiniowanego podobieństwa cosinusowego jest zdefiniowana jako:

$$d_C(\mathbf{A}, \mathbf{B}) = 1 - S_C(\mathbf{A}, \mathbf{B})$$


### Miara chi-kwadrat (chi-square)
Miara chi-kwadrat jest stosowana do porównywania rozkładów danych. Jest szczególnie użyteczna w przypadku danych kategorycznych i histogramów. Porównuje ona różnice między obserwowanymi a oczekiwanymi wartościami cech, normalizując je przez ich sumę.


Ogólnie, w przestrzeni $n$-wymiarowej, dla punktów o współrzędnych $P = (p_1, p_2, ..., p_n)$ i $Q = (q_1, q_2, ..., q_n)$, odległość chi-kwadrat wyraża się wzorem:

$$d_{\chi^2}(\mathbf{P}, \mathbf{Q}) = \sum_{i=1}^n \frac{(p_i - q_i)^2}{p_i+q_i} $$

W celu stworzenia modelu korzystamy z następujących bibliotek:
- `pandas` - służy do analizy, czyszczenia, manipulowania danymi.
- `seaborn` i `matplotlib` - służą do wizualizowania danych.
- `numpy` - służy do przetwarzania danych numerycznych i wykonywania obliczeń potrzebnych do analizy filmów.
- `ipywidgets` i `display` - służą do utworzenia interfejsu przyjaznego dla użytkownika

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from IPython.display import display, HTML

### Zbiór danych
Zbiór danych zawiera informacje na temat filmów z różnych lat, gatunków i krajów. Składa się z 19 kolumn, z których każda opisuje różne aspekty każdego filmu.


- filmtv_id - unikalny identyfikator
- title - tytuł 
- year - rok wydania 
- genre - gatunek 
- duration - czas trwania 
- country - kraj produkcji 
- directors - reżyserzy 
- actress - aktorzy 
- avg_vote -średnia ocena filmu.
- critics_vote - ocena  przez krytyków.
- public_vote - ocena  przez publiczność
- total_votes - liczba głosów oddanych na film
- description - opis 
- notes - dodatkowe notatki 
- humor - ocena humoru 
- rhythm - ocena tempa 
- effort - ocena wysiłku włożonego w produkcję 
- tension - ocena napięcia 
- erotism - ocena elementów erotycznych  

In [ ]:
Data_frame_base = pd.read_csv(r'filmtv_movies.csv')
Data_frame_base.head()

### Wstępna analiza danych 

####  Histogram roczników filmów

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(Data_frame_base['year'], bins=30, kde=True)
plt.title('Histogram roczników filmów')
plt.xlabel('Rok')
plt.ylabel('Liczba filmów')
plt.show()

Znacząca część filmów w zbiorze danych to filmy z 2000 roku i wyżej.

#### Rozkład ocen filmów

In [ ]:

plt.figure(figsize=(10, 6))
sns.histplot(Data_frame_base['avg_vote'], bins=30, kde=True)
plt.title('Rozkład średnich ocen filmów')
plt.xlabel('Średnia ocena')
plt.ylabel('Liczba filmów')
plt.show()

Większość filmów otrzymuje oceny w zakresie od 5 do 7, co sugeruje, że średnia ocena filmów w tym zbiorze wynosi około 6. Bardzo mało filmów jest ocenianych 10/10.

#### Rozkład gatunków filmowych

In [ ]:
plt.figure(figsize=(14, 8))
sns.countplot(y='genre', data=Data_frame_base, order=Data_frame_base['genre'].value_counts().index, palette='coolwarm')
plt.title('Rozkład gatunków filmowych')
plt.xlabel('Liczba filmów')
plt.ylabel('Gatunek')
plt.show()

W zbiorze danych przewagę mają filmy o gatunku drama oraz komedie, daleko za nimi mamy thrilley, horrory czy filmy akcji.

#### Średnia ocena dla każdego gatunku

In [ ]:
avg_vote_by_genre = Data_frame_base.groupby('genre')['avg_vote'].mean().reset_index()

plt.figure(figsize=(12, 8))
sns.barplot(x='avg_vote', y='genre', data=avg_vote_by_genre, palette='coolwarm')
plt.xlabel('Średnia ocena')
plt.ylabel('Gatunek')
plt.title('Średnia ocena filmów według gatunku')
plt.show()

Widzimy, że najwyżej ocenianym gatunkiem są komedie stand-up, a najniżej filmy erotyczne.

#### Średnia długość filmów według gatunku

In [ ]:
avg_duration_by_genre = Data_frame_base.groupby('genre')['duration'].mean().reset_index()

plt.figure(figsize=(12, 8))
sns.barplot(x='duration', y='genre', data=avg_duration_by_genre, palette='coolwarm')
plt.xlabel('Średnia długość filmu (minuty)')
plt.ylabel('Gatunek')
plt.title('Średnia długość filmów według gatunku')
plt.show()

Z wykresu wynika, że długość filmów różni się znacznie w zależności od gatunku. Najdłuższe filmy to "Short Movie" z średnią długością przekraczającą 200 minut, podczas gdy najkrótsze to "Stand-up Comedy", które mają średnią długość poniżej 50 minut. Większość gatunków filmowych mieści się w przedziale od 75 do 125 minut.

#### Wykres mapy pustych kolumn

In [ ]:
plt.figure(figsize=(30, 12))
sns.heatmap(Data_frame_base.isnull(),yticklabels=False, cbar=False, cmap='coolwarm')
plt.title('Puste rekordy w kolumnach (heatmap)')
plt.xlabel('Kolumny')
plt.ylabel('Puste rekordy')
plt.show()

Kolumny `directors`, `actors`, `critics_vote` i `notes` mają znaczną liczbę brakujących wartości. W szczególności kolumna `notes` ma wiele brakujących danych, co sugeruje, że dodatkowe notatki na temat filmów nie są dostępne dla wielu rekordów. Pozostałe kolumny  mają niewiele lub brak brakujących wartości.

### Przekształcanie danych

Przerobimy teraz nasz zbiór danych w taki sposób, że kolumna "directors" zawierająca informacje o reżyserach będzie są przekształcana z pojedynczych ciągów znaków zawierających listę elementów oddzielonych przecinkami, na listy Pythona. Jeśli kolumna zawiera wartość NaN, zostaje zastąpiona zerem. Kolumny zawierające wartości logiczne są przekształcane na wartości całkowitoliczbowe.

In [ ]:
def directors_and_actors_split(df):

    df = df.fillna(0)

    for col in df.columns:
        if df[col].dtype == 'bool':
            df[col] = df[col].astype(int)


    for i in range(df.shape[0]):
        if df['directors'][i] != 0:
           df['directors'][i] = df['directors'][i].split(', ')
        if df['actors'][i] != 0:
           df['actors'][i] = df['actors'][i].split(', ')
    return(df)

Data_frame = directors_and_actors_split(Data_frame_base)

Data_frame.head()

Zdefiniujmy funkcję `preprocessing`, która przekształca ramkę danych, tworząc zmienne kategoryczne dla wybranych kolumn, usuwa określone kolumny oraz wypełnia brakujące wartości zerami. Dodatkowo, konwertuje wartości logiczne na wartości całkowitoliczbowe dla spójności danych. Ostatecznie zwraca przetworzoną ramkę danych gotową do dalszej analizy lub modelowania

In [ ]:
def preprocessing(df,c_dummy,c_drop):

    df_dummy = pd.get_dummies(Data_frame, columns=c_dummy)

    df_base = df_dummy.drop(c_drop, axis = 1)

    df_nulls = df_base.fillna(0)

    for col in df_nulls.columns:
        if df_nulls[col].dtype == 'bool':
            df_nulls[col] = df_nulls[col].astype(int)

    df_preprocessed = df_nulls

    return df_preprocessed

columns_dummy=['genre','country']
columns_drop = ['notes','description','filmtv_id']

Data_frame_prep = preprocessing(Data_frame, columns_dummy, columns_drop)

Data_frame_prep.head()

### Funkcje liczące miary podobieństwa

Zdefiniujmy funkcje liczące miary podobieństwa:

Przypomnijmy:

##### Odległość Euklidesowa:
$$ d_E(\mathbf{P}, \mathbf{Q}) = \sqrt{\sum_{i=1}^{n} (p_i - q_i)^2} $$

In [ ]:
def Euclid_distance(x,y):
    if np.all(x == y):
        dist = np.linalg.norm(y)
    else:
        dist = np.sqrt(np.sum((x-y)**2))
    return dist

##### Odległość Cosinusowa:
$$S_C(\mathbf{A}, \mathbf{B}) = \frac{\mathbf{A} \cdot \mathbf{B}}{\|\mathbf{A}\| \|\mathbf{B}\|} $$

$$d_C(\mathbf{A}, \mathbf{B}) = 1 - S_C(\mathbf{A}, \mathbf{B})$$

In [ ]:
def Cosine_distance(x,y):
    if np.all(x == y):
        similarity = -1
    else:
        similarity = np.dot(x,y)/(np.linalg.norm(x)*np.linalg.norm(y))
    distance = 1 - similarity
    return distance

##### Odległość Chi-Kwadrat

$$d_{\chi^2}(\mathbf{P}, \mathbf{Q}) = \sum_{i=1}^n \frac{(p_i - q_i)^2}{p_i+q_i} $$

In [ ]:
def Chi_square_distance(x,y):
    epsilon = 1e-10
    if np.all(x == y):
        distance = np.linalg.norm(y)
    else:
        distance = np.sum((x-y)**2/(x+y+epsilon))
    return distance

Za pomocą zdefiniowanej wcześniej funkcji `preprocessing` zamieniamy zmienne z kolumn "genre" i "country"  na zmienne kategoryczne, natomiast zmienne z kolumn "directors", "actors", "notes", "description" i "filmtv_id" usuwamy.

In [ ]:
columns_dummy=['genre','country']
columns_drop = ['directors','actors','notes','description','filmtv_id']

Data_frame_input = preprocessing(Data_frame, columns_dummy, columns_drop)

In [ ]:
Data_frame_input

### Pierwszy model rekomendacji filmów

Zdefiniujmy funkcję `recommend_me_some_films`, która jest narzędziem do rekomendacji filmów na podstawie różnych miar odległości oraz dodatkowych kryteriów takich jak rok produkcji i czas trwania filmu

Kroki, które wykonuje funkcja:
1. Normalizuje kolumny 'year' i 'duration', jeśli wybrano 'alternative'.
2. Prosi użytkownika o podanie tytułu filmu.
3. Porównuje wybrany film z innymi, używając wybranej miary odległości.
4. Sortuje filmy według odległości i zwraca n najbliższych filmów.
5. Tworzy DataFrame z rekomendowanymi tytułami i oryginalnym filmem.

In [ ]:
def recommend_me_some_films(df, n = 1, measure = 'Euclid', year_importance = 'classic', duration_importance = 'classic'):

    if year_importance == 'alternative':
        df['year'] = (df['year'] - df['year'].min())/10

    if duration_importance == 'alternative':
        df['duration'] = (df['duration']- df['duration'].min())/100

    Y = df['title'].to_numpy()
    X = df.drop('title', axis=1).to_numpy()

    film = input("Podaj tytuł filmu :")

    Y_index = np.where(Y == film)[0][0]

    film_vector = X[Y_index,:]

    l = X.shape[0]

    list_distances = []

    if measure == 'Mixed':
        list_distances_E = []
        list_distances_C = []
        list_distances_CS = []
        for i in range(l):
            vector = X[i,:]
            distance_E = Euclid_distance(film_vector, vector)
            distance_C = Cosine_distance(film_vector, vector)
            distance_CS = Chi_square_distance(film_vector, vector)
            list_distances_E.append(distance_E)
            list_distances_C.append(distance_C)
            list_distances_CS.append(distance_CS)

        distances_vector_E = np.array(list_distances_E).reshape(-1,1)
        distances_vector_C = np.array(list_distances_C).reshape(-1,1)
        distances_vector_CS = np.array(list_distances_CS).reshape(-1,1)

        Data = np.hstack([X,distances_vector_E, distances_vector_C, distances_vector_CS])

        titles = Y.reshape(-1,1)

        Data_title = np.hstack([titles,Data])

        indeksy_sortujace_E = np.argsort(Data_title[:, -3])
        indeksy_sortujace_C = np.argsort(Data_title[:, -2])
        indeksy_sortujace_CS = np.argsort(Data_title[:, -1])

        macierz_posortowana_E = Data_title[indeksy_sortujace_E]
        macierz_posortowana_C = Data_title[indeksy_sortujace_C]
        macierz_posortowana_CS = Data_title[indeksy_sortujace_CS]
        
        
        nearest_titles_E = macierz_posortowana_E[:n,0]
        nearest_titles_C = macierz_posortowana_C[:n,0]
        nearest_titles_CS = macierz_posortowana_CS[:n,0]

        titles_list = (nearest_titles_E, nearest_titles_C, nearest_titles_CS)

        combined_list = []
        for titles in titles_list:
            combined_list.extend(titles)

        unique_titles = list(set(combined_list))

        df_titles_mixed = pd.DataFrame(unique_titles,columns=["Titles"])
        df_titles_mixed['Chosen_film'] = ''
        df_titles_mixed.loc[0, 'Chosen_film'] = film

        return (df_titles_mixed)

    else:
        for i in range(l):
            vector  = X[i,:]

            if measure == 'Cosine':
                distance = Cosine_distance(film_vector,vector)
            elif measure == 'Chi_square':
                distance = Chi_square_distance(film_vector,vector)
            elif measure == 'Euclid':
                distance = Euclid_distance(film_vector,vector)

            list_distances.append(distance)

        distances_vector = np.array(list_distances).reshape(-1,1)
        Data = np.hstack([X,distances_vector])
        titles = Y.reshape(-1,1)
        Data_title = np.hstack([titles,Data])

        indeksy_sortujace = np.argsort(Data_title[:, -1])

        macierz_posortowana = Data_title[indeksy_sortujace]
        
        nearest_titles = macierz_posortowana[:n,0]

        df_titles = pd.DataFrame(nearest_titles,columns=["Titles"])
        df_titles['Chosen_film'] = ''
        df_titles.loc[0, 'Chosen_film'] = film
        return (df_titles)

Aby przetestować model rekomendacji filmów znajdziemy teraz filmy podobne do Interstellar, oraz Braveheart.
W tym celu należy wywołać poniższy kod oraz podać film.

In [ ]:
recommend_me_some_films(Data_frame_input, n = 5, measure = 'Mixed', year_importance = 'classic', duration_importance='classic')

In [ ]:
recommend_me_some_films(Data_frame_input, n=5, measure='Mixed', year_importance='alternative', duration_importance='alternative')

### Drugi model rekomendacji filmów

Zdefiniujmy kolejną funkcję `recommend_me_some_films_d`, która także rekomenduje filmy, uwzględniając dodatkowo reżysera jako kryterium podobieństwa.

Kroki, które wykonuje funkcja:
1. Normalizuje kolumny 'year' i 'duration', jeśli wybrano 'alternative'.
2. Jeśli wybrano 'alternative' dla reżyserów, uwzględnia ich wpływ na podobieństwo filmów.
3. Prosi użytkownika o podanie tytułu filmu.
4. Porównuje wybrany film z innymi, używając wybranej miary odległości.
5. Sortuje filmy według odległości i zwraca n najbliższych filmów.
6. Tworzy DataFrame z rekomendowanymi tytułami i oryginalnym filmem.


In [ ]:
def recommend_me_some_films_d(df, n = 1, measure = 'Euclid', year_importance = 'classic', duration_importance = 'classic', directors_importance = 'classic'):

    if directors_importance == 'classic':
        df.drop('directors', axis = 1)

    if year_importance == 'alternative':
        df['year'] = (df['year'] - df['year'].min())/10

    if duration_importance == 'alternative':
        df['duration'] = (df['duration']- df['duration'].min())/100

    Y = df['title'].to_numpy()
    X = df.drop('title', axis=1).to_numpy()

    film = input("Podaj tytuł filmu :")

    Y_index = np.where(Y == film)[0][0]

    film_vector = X[Y_index,:]

    l = X.shape[0]

    if directors_importance == 'alternative':
        film_vector = np.hstack((film_vector, 0))
        result = []
        for i in range(l):
            vector = X[i, :]
            if isinstance(film_vector[2], int) or isinstance(vector[2], int):
                director_measure = 15

            else:
                director_measure = 0
                for j in range(len(film_vector[2])):
                    for k in range(len(vector[2])):
                        if film_vector[2][j] != vector[2][k]:
                            director_measure += 5
            
            result.append(np.hstack((X[i], director_measure)))

        X = np.array(result, dtype=object)

        columns_to_remove = [2]

        X = np.delete(X, columns_to_remove, axis=1)
        film_vector = np.delete(film_vector, columns_to_remove)

    list_distances = []

    if measure == 'Mixed':
        list_distances_E = []
        list_distances_C = []
        list_distances_CS = []
        for i in range(l):
            vector = X[i,:]
            distance_E = Euclid_distance(film_vector, vector)
            distance_C = Cosine_distance(film_vector, vector)
            distance_CS = Chi_square_distance(film_vector, vector)
            list_distances_E.append(distance_E)
            list_distances_C.append(distance_C)
            list_distances_CS.append(distance_CS)

        distances_vector_E = np.array(list_distances_E).reshape(-1,1)
        distances_vector_C = np.array(list_distances_C).reshape(-1,1)
        distances_vector_CS = np.array(list_distances_CS).reshape(-1,1)

        Data = np.hstack([X,distances_vector_E, distances_vector_C, distances_vector_CS])

        titles = Y.reshape(-1,1)

        Data_title = np.hstack([titles,Data])

        indeksy_sortujace_E = np.argsort(Data_title[:, -3])
        indeksy_sortujace_C = np.argsort(Data_title[:, -2])
        indeksy_sortujace_CS = np.argsort(Data_title[:, -1])

        macierz_posortowana_E = Data_title[indeksy_sortujace_E]
        macierz_posortowana_C = Data_title[indeksy_sortujace_C]
        macierz_posortowana_CS = Data_title[indeksy_sortujace_CS]
        
        
        nearest_titles_E = macierz_posortowana_E[:n,0]
        nearest_titles_C = macierz_posortowana_C[:n,0]
        nearest_titles_CS = macierz_posortowana_CS[:n,0]

        titles_list = (nearest_titles_E, nearest_titles_C, nearest_titles_CS)

        combined_list = []
        for titles in titles_list:
            combined_list.extend(titles)

        unique_titles = list(set(combined_list))

        df_titles_mixed = pd.DataFrame(unique_titles,columns=["Titles"])
        df_titles_mixed['Chosen_film'] = ''
        df_titles_mixed.loc[0, 'Chosen_film'] = film

        return (df_titles_mixed)

    else:
        for i in range(l):
            vector  = X[i,:]

            if measure == 'Cosine':
                distance = Cosine_distance(film_vector,vector)
            elif measure == 'Chi_square':
                distance = Chi_square_distance(film_vector,vector)
            elif measure == 'Euclid':
                distance = Euclid_distance(film_vector,vector)

            list_distances.append(distance)

        distances_vector = np.array(list_distances).reshape(-1,1)
        Data = np.hstack([X,distances_vector])
        titles = Y.reshape(-1,1)
        Data_title = np.hstack([titles,Data])

        indeksy_sortujace = np.argsort(Data_title[:, -1])

        macierz_posortowana = Data_title[indeksy_sortujace]
        
        nearest_titles = macierz_posortowana[:n,0]

        df_titles = pd.DataFrame(nearest_titles,columns=["Titles"])
        df_titles['Chosen_film'] = '' 
        df_titles.loc[0, 'Chosen_film'] = film
        
        return (df_titles)

W poprzednim modelu usuwaliśmy kolumnę z reżyserami, tym razem zostawimy ją i na takich danych przetestujemy nasz rekomendator.

In [ ]:
columns_dummy=['genre','country']
columns_drop = ['actors','notes','description','filmtv_id']

Data_frame_d = preprocessing(Data_frame,columns_dummy,columns_drop)

Data_frame_d.head()

In [ ]:
recommend_me_some_films_d(Data_frame_d, n=5, measure='Mixed', directors_importance='alternative')

In [ ]:
recommend_me_some_films_d(Data_frame_d, n=5, measure='Mixed', year_importance='alternative', duration_importance = 'alternative' ,directors_importance='alternative')

### Model z przyjaznym dla użytkownika interfejsem

Musimy zmodyfikować naszą poprzednią funkcję, aby inaczej przekazywać do niej wybrany przez nas tytuł filmu.

In [ ]:
def recommend_me_some_films_d_m(df, film, n=1, measure='Euclid', year_importance='classic', duration_importance='classic', directors_importance='classic'):
    if directors_importance == 'classic':
        df = df.drop('directors', axis=1)

    if year_importance == 'alternative':
        df['year'] = (df['year'] - df['year'].min()) / 10

    if duration_importance == 'alternative':
        df['duration'] = (df['duration'] - df['duration'].min()) / 100

    Y = df['title'].to_numpy()
    X = df.drop('title', axis=1).to_numpy()

    Y_index = np.where(Y == film)[0][0]
    film_vector = X[Y_index, :]

    l = X.shape[0]

    if directors_importance == 'alternative':
        film_vector = np.hstack((film_vector, 0))
        result = []
        for i in range(l):
            vector = X[i, :]
            if isinstance(film_vector[2], int) or isinstance(vector[2], int):
                director_measure = 15
            else:
                director_measure = 0
                for j in range(len(film_vector[2])):
                    for k in range(len(vector[2])):
                        if film_vector[2][j] != vector[2][k]:
                            director_measure += 5

            result.append(np.hstack((X[i], director_measure)))

        X = np.array(result, dtype=object)
        columns_to_remove = [2]
        X = np.delete(X, columns_to_remove, axis=1)
        film_vector = np.delete(film_vector, columns_to_remove)

    list_distances = []

    if measure == 'Mixed':
        list_distances_E = []
        list_distances_C = []
        list_distances_CS = []
        for i in range(l):
            vector = X[i, :]
            distance_E = Euclid_distance(film_vector, vector)
            distance_C = Cosine_distance(film_vector, vector)
            distance_CS = Chi_square_distance(film_vector, vector)
            list_distances_E.append(distance_E)
            list_distances_C.append(distance_C)
            list_distances_CS.append(distance_CS)

        distances_vector_E = np.array(list_distances_E).reshape(-1, 1)
        distances_vector_C = np.array(list_distances_C).reshape(-1, 1)
        distances_vector_CS = np.array(list_distances_CS).reshape(-1, 1)

        Data = np.hstack([X, distances_vector_E, distances_vector_C, distances_vector_CS])

        titles = Y.reshape(-1, 1)
        Data_title = np.hstack([titles, Data])

        indeksy_sortujace_E = np.argsort(Data_title[:, -3])
        indeksy_sortujace_C = np.argsort(Data_title[:, -2])
        indeksy_sortujace_CS = np.argsort(Data_title[:, -1])

        macierz_posortowana_E = Data_title[indeksy_sortujace_E]
        macierz_posortowana_C = Data_title[indeksy_sortujace_C]
        macierz_posortowana_CS = Data_title[indeksy_sortujace_CS]

        nearest_titles_E = macierz_posortowana_E[:n, 0]
        nearest_titles_C = macierz_posortowana_C[:n, 0]
        nearest_titles_CS = macierz_posortowana_CS[:n, 0]

        titles_list = (nearest_titles_E, nearest_titles_C, nearest_titles_CS)

        combined_list = []
        for titles in titles_list:
            combined_list.extend(titles)

        unique_titles = list(set(combined_list))

        df_titles_mixed = pd.DataFrame(unique_titles, columns=["Titles"])
        df_titles_mixed['Chosen_film'] = ''
        df_titles_mixed.loc[0, 'Chosen_film'] = film

        return df_titles_mixed

    else:
        for i in range(l):
            vector = X[i, :]

            if measure == 'Cosine':
                distance = Cosine_distance(film_vector, vector)
            elif measure == 'Chi_square':
                distance = Chi_square_distance(film_vector, vector)
            elif measure == 'Euclid':
                distance = Euclid_distance(film_vector, vector)

            list_distances.append(distance)

        distances_vector = np.array(list_distances).reshape(-1, 1)
        Data = np.hstack([X, distances_vector])
        titles = Y.reshape(-1, 1)
        Data_title = np.hstack([titles, Data])

        indeksy_sortujace = np.argsort(Data_title[:, -1])

        macierz_posortowana = Data_title[indeksy_sortujace]

        nearest_titles = macierz_posortowana[:n, 0]

        df_titles = pd.DataFrame(nearest_titles, columns=["Titles"])
        df_titles['Chosen_film'] = ''
        df_titles.loc[0, 'Chosen_film'] = film

        return df_titles

In [ ]:
columns_dummy=['genre','country']
columns_drop = ['actors','notes','description','filmtv_id']

Data_frame_d_i = preprocessing(Data_frame,columns_dummy,columns_drop)

Data_frame_d_i.head()

### Wywołujemy 

In [ ]:
film_input = widgets.Text(
    value='',
    placeholder='Wprowadź tytuł filmu',
    description='Film:',
    disabled=False,
    layout=widgets.Layout(width='400px', margin='10px 0px')
)

n_input = widgets.IntText(
    value=1,
    description='Liczba n:',
    disabled=False,
    layout=widgets.Layout(width='200px', margin='10px 0px')
)

measure_dropdown = widgets.Dropdown(
    options=['Euclid', 'Cosine', 'Chi_square', 'Mixed'],
    value='Euclid',
    description='Miara:',
    layout=widgets.Layout(width='300px', margin='10px 0px')
)

year_importance_dropdown = widgets.Dropdown(
    options=['classic', 'alternative'],
    value='classic',
    description='Rok:',
    layout=widgets.Layout(width='300px', margin='10px 0px')
)

duration_importance_dropdown = widgets.Dropdown(
    options=['classic', 'alternative'],
    value='classic',
    description='Czas trwania:',
    layout=widgets.Layout(width='300px', margin='10px 0px')
)

directors_importance_dropdown = widgets.Dropdown(
    options=['classic', 'alternative'],
    value='classic',
    description='Reżyserzy:',
    layout=widgets.Layout(width='300px', margin='10px 0px')
)

button = widgets.Button(
    description='Znajdź podobne:',
    disabled=False,
    button_style='info',
    tooltip='Kliknij, aby wysłać',
    icon='search',
    layout=widgets.Layout(width='200px', margin='20px 0px')
)

output = widgets.Output()

def on_button_clicked(b):
    with output:
        output.clear_output()
        film = film_input.value
        n = n_input.value
        measure = measure_dropdown.value
        year_importance = year_importance_dropdown.value
        duration_importance = duration_importance_dropdown.value
        directors_importance = directors_importance_dropdown.value
        
        result = recommend_me_some_films_d_m(Data_frame_d_i, film, n, measure, year_importance, duration_importance, directors_importance)
        display(result)

button.on_click(on_button_clicked)

display(HTML("<style>.widget-inline-hbox > * { margin: 10px; } .widget-label { min-width: 100px; }</style>"))


widgets.VBox([
    widgets.HBox([film_input, n_input]),
    widgets.HBox([measure_dropdown]),
    widgets.HBox([year_importance_dropdown]),
    widgets.HBox([duration_importance_dropdown]),
    widgets.HBox([directors_importance_dropdown]),
    button,
    output
])